In [27]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tabulate import tabulate
import tensorflow as tf
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

In [29]:
df=pd.read_csv("Wine.csv")
df.head()

,Class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [31]:
X = df.drop("Class", axis=1).values
y = df["Class"].values - 1 

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_train_oh = np.eye(3)[y_train]  

In [33]:
start_tf = time.time()

model_tf = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(13,)),
    tf.keras.layers.Dense(3, activation='softmax')
])
model_tf.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model_tf.fit(X_train, y_train_oh, epochs=10, batch_size=16, verbose=0)

loss_tf, acc_tf = model_tf.evaluate(X_test, np.eye(3)[y_test], verbose=0)
end_tf = time.time()
time_tf = end_tf - start_tf

C:\Users\lenovo\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [34]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=16, shuffle=True)

In [37]:
class WineNet(nn.Module):
    def __init__(self):
        super(WineNet, self).__init__()
        self.fc1 = nn.Linear(13, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 3)

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

model_pt = WineNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_pt.parameters(), lr=0.001)

start_pt = time.time()

In [39]:
for epoch in range(10):
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()
        output = model_pt(batch_x)
        loss = criterion(output, batch_y)
        loss.backward()
        optimizer.step()

with torch.no_grad():
    test_output = model_pt(X_test_tensor)
    _, preds_pt = torch.max(test_output, 1)
    acc_pt = accuracy_score(y_test_tensor, preds_pt)

end_pt = time.time()
time_pt = end_pt - start_pt


In [41]:
results = pd.DataFrame({
    "Framework": ["TensorFlow", "PyTorch"],
    "Accuracy": [f"{acc_tf:.4f}", f"{acc_pt:.4f}"],
    "Training Time (s)": [f"{time_tf:.2f}", f"{time_pt:.2f}"]
})

print(" Framework Benchmark Results:\n")
print(tabulate(results, headers="keys", tablefmt="fancy_grid"))

 Framework Benchmark Results:

╒════╤═════════════╤════════════╤═════════════════════╕
│    │ Framework   │   Accuracy │   Training Time (s) │
╞════╪═════════════╪════════════╪═════════════════════╡
│  0 │ TensorFlow  │          1 │                1    │
├────┼─────────────┼────────────┼─────────────────────┤
│  1 │ PyTorch     │          1 │                0.62 │
╘════╧═════════════╧════════════╧═════════════════════╛
